In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('./src')
import warnings
warnings.filterwarnings("ignore")

from data import PPCI

# Results

- **Encoder**: clip, clip_large, dino, mae, vit, vit_large
- **Token**: class, mean, all
- **Splitting criteria**: experiment, experiment_easy, position, position_easy, random, random_easy
- **Task**: all, or
- **Hidden Layers**: 1, 2
- **Learning Rate**: 0.1, 0.01, 0.001
- **Seed** = 0, 1, 2, 3, 4

In [ ]:
results = pd.read_csv("/nfs/scistore19/locatgrp/rcadei/ISTAnt/results/istant_lq/experiments_result.csv", index_col=0)
results[results.train == True].drop(["tr_equality_control", "tr_equality_treatment", "seed", "encoder"], axis=1).head(20)

# results = results.dropna()
# results = results.set_index(["encoder", "token", "task", "lr", "seed"])
# results = results.groupby(["encoder", "token","task", "lr"]).mean()
# results = results.sort_values("val_accuracy", ascending=False)
# results = results.xs("all", level="task")
# results.head(20)

In [ ]:
dataset = PPCI(encoder = "dino",
               token = "class",
               task = "all",
               split_criteria = "experiment",
               environment = "supervised",
               #batch_size = 256,
               num_proc = 4,
               verbose = True)
# dataset.plot_out_distribution()
dataset.train( add_pred_env="supervised", 
              hidden_layers = 1,
              hidden_nodes = 256,
              batch_size = 256,
              lr = 0.005,
              seed = 0,
              num_epochs=10,
              save = True,
              verbose=True)
# dataset.visualize(k=8, save=True)
# result_tr_y = dataset.evaluate(color="yellow", train=True, verbose=False)
# result_tr_b = dataset.evaluate(color="blue", train=True, verbose=False)
# result_val_y = dataset.evaluate(color="yellow", train=False, verbose=False)
# result_val_b = dataset.evaluate(color="blue", train=False, verbose=False)

In [ ]:
dataset.supervised["Y"][:,1].mean()

In [ ]:
dataset.evaluate(color="blue", train=True, verbose=False)

In [ ]:
dataset.supervised["Y_hat"]

In [ ]:
from causal import compute_ate
compute_ate(dataset.supervised["Y_hat"], 
            dataset.supervised["T"], 
            dataset.supervised["W"], 
            method="ead", 
            color="blue")

In [ ]:
# dataset = PPCI()
# dataset.plot_out_distribution()
# dataset.train()
# dataset.visualize()
# dataset.evaluate()

## Post-Processing

In [ ]:
exp = (dataset.supervised["source_data"]["experiment"]==4)
pos = (dataset.supervised["source_data"]["position"]==1)
filter = (exp & pos).nonzero().squeeze()
y = dataset.supervised["Y"][filter][:,0].detach()
y_hat = dataset.supervised["Y_hat"][filter][:,0].detach()
y_pred = y_hat.round()

plt.scatter(range(len(filter)), y_hat, s=1, c="blue", alpha=0.5, label="y_probs")
plt.scatter(range(len(filter)),y_pred-(-1)**y_pred.detach()*0.04, s=1, c="red", alpha=0.5, label="y_pred")
plt.scatter(range(len(filter)), y-(-1)**y.detach()*0.02, s=1, c="green", alpha=0.5, label="y")
plt.legend()
plt.show()

In [ ]:
frame = (dataset.supervised["source_data"]["frame"]==2220)
idx = (exp & pos & frame).nonzero().item()
img = dataset.supervised["source_data"][idx]["image"]
outcome = dataset.supervised["source_data"][idx]["outcome"]

img = img.permute(1, 2, 0)
plt.title(f"Y2F: {int(outcome[0])}, B2F: {int(outcome[1])}")
plt.imshow(img);